In [1]:
import keras
import now as now
import numpy as np
import tensorflow
import matplotlib.pyplot as plt
import segmentation_models as sm
import tf as tf
from Modelling.Baselines.Data_Preprocessing import import_labeled_data, import_labeled_photos
import os
import albumentations as A

Segmentation Models: using `keras` framework.


In [ ]:

def get_training_augmentation():
    train_transform = [

        A.HorizontalFlip(p=0.5),

        A.ShiftScaleRotate(scale_limit=0.5, rotate_limit=0, shift_limit=0.1, p=1, border_mode=0),

        A.PadIfNeeded(min_height=220, min_width=220, always_apply=True, border_mode=0),
        A.RandomCrop(height=220, width=220, always_apply=True),

        A.IAAAdditiveGaussianNoise(p=0.2),
        A.IAAPerspective(p=0.5),

        A.OneOf(
            [
                A.CLAHE(p=1),
                A.RandomBrightness(p=1),
                A.RandomGamma(p=1),
            ],
            p=0.9,
        ),

        A.OneOf(
            [
                A.IAASharpen(p=1),
                A.Blur(blur_limit=3, p=1),
                A.MotionBlur(blur_limit=3, p=1),
            ],
            p=0.9,
        ),

        A.OneOf(
            [
                A.RandomContrast(p=1),
                A.HueSaturationValue(p=1),
            ],
            p=0.9,
        ),
        A.Lambda(mask=round_clip_0_1)
    ]
    return A.Compose(train_transform)

In [14]:
from Modelling.Baselines.Data_Preprocessing import import_labeled_data, import_labeled_photos


x, y = import_labeled_photos(bands=slice(0, 108))





0
1
2
3
4
5
6
7
8
9
10
11
12
13


In [2]:
class Dataloder(keras.utils.Sequence):
    """Load data from dataset and form batches

    Args:
        dataset: instance of Dataset class for image loading and preprocessing.
        batch_size: Integet number of images in batch.
        shuffle: Boolean, if `True` shuffle image indexes each epoch.
    """

    def __init__(self, dataset, batch_size=1, shuffle=False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(dataset))

        self.on_epoch_end()

    def __getitem__(self, i):

        # collect batch data
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])

        # transpose list of lists
        batch = [np.stack(samples, axis=0) for samples in zip(*data)]

        return batch

    def __len__(self):
        """Denotes the number of batches per epoch"""
        return len(self.indexes) // self.batch_size

    def on_epoch_end(self):
        """Callback function to shuffle indexes each epoch"""
        if self.shuffle:
            self.indexes = np.random.permutation(self.indexes)

### Import Data

In [3]:
from Modelling.Baselines.Data_Preprocessing import import_labeled_data,import_labeled_photos

#from Modelling.Baselines.Data_Preprocessing import import_labeled_data,import_labeled_photos



X,Y=import_labeled_photos()

In [3]:
a=X[0]
a

NameError: name 'X' is not defined

In [5]:
from numpy.random.mtrand import random

def Split_Train_Test_Photos(X,Y):
    frac = 0.2  # how much of a/b do you want to exclude
    # generate a list of indices to exclude. Turn in into a set for O(1) lookup time
    #inds_x = set(random.sample(list(range(len(X))), int(frac*len(X))))
    #inds_y = set(random.sample(list(range(len(X))), int(frac*len(X))))
    
    # use `enumerate` to get list indices as well as elements. 
    # Filter by index, but take only the elements
    #x_val = [n for i,n in enumerate(X) if i not in inds_x]
    #x_train = [n for i,n in enumerate(X) if i in inds_x]
    #y_val = [n for i,n in enumerate(Y) if i not in inds_y]
    #y_train = [n for i,n in enumerate(Y) if i in inds_y]
    x_val = X[1]
    x_train = X[0]
    y_val = Y[1]
    y_train = Y[0]

    return x_train,x_val,y_train,y_val

In [10]:
from Modelling.Baselines.Data_Preprocessing import import_labeled_photos
from sklearn.model_selection import train_test_split

BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)

# load your data
X, Y= import_labeled_photos()
#x_train,x_val,y_train,y_val=Split_Train_Test_Photos(X,Y)
x_train,x_val,y_train,y_val=train_test_split(X,Y)
# preprocess input
x_train = preprocess_input(x_train)[0]
x_val = preprocess_input(x_val)[0]
y_train=y_train[0]
y_val=y_val[0]

from segmentation_models import Unet

# read/scale/preprocess data


# define number of channels
N = X[0].shape[-1]
print(N)

# define model
model = sm.Unet(backbone_name='resnet34', encoder_weights=None, input_shape=(None, None, N))


model.compile(
    'Adam',
    loss=sm.losses.bce_jaccard_loss,
    metrics=[sm.metrics.iou_score],
)

# fit model
# if you use data generator use model.fit_generator(...) instead of model.fit(...)
# more about `fit_generator` here: https://keras.io/models/sequential/#fit_generator


109
109


In [11]:
x_train.shape

(31, 31, 109)

In [13]:
from keras.utils.version_utils import callbacks
from keras.integration_test.preprocessing_test_utils import BATCH_SIZE
from tensorflow.python.data import Dataset
import tensorflow as tf
CLASSES = ['None','Wiese','Straße','Auto','See','Schienen','Haus','Wald']
#labels = {0:'None',1:'Wiese', 2:'Straße',3: 'Auto', 4:'See', 5:'Schienen', 6:'Haus', 7:'Wald'}

LR = 0.0001
EPOCHS = 40
# train model
train_dataset =tf.data.Dataset.from_tensor_slices((x_train,y_train))
valid_dataset =tf.data.Dataset.from_tensor_slices(x_val,y_val)

train_dataloader = Dataloder(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_dataloader = Dataloder(valid_dataset, batch_size=1, shuffle=False)

# check shapes for errors
assert train_dataloader[0][0].shape == (BATCH_SIZE, 32, 32, 109)
assert train_dataloader[0][1].shape == (BATCH_SIZE, 32, 32, 7)

history = model.fit_generator(
    train_dataloader,
    steps_per_epoch=len(train_dataloader),
    epochs=EPOCHS,
    callbacks=callbacks,
    validation_data=valid_dataloader,
    validation_steps=len(valid_dataloader),
)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
history=model.fit(
   x=x_train[0],
   y=y_train[0],
   epochs=10,
   validation_data=(x_val[0], y_val[0]),
)

In [ ]:
# Plot training & validation iou_score values
plt.figure(figsize=(30, 5))
plt.subplot(121)
plt.plot(history.history['iou_score'])
plt.plot(history.history['val_iou_score'])
plt.title('Model iou_score')
plt.ylabel('iou_score')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

# Plot training & validation loss values
plt.subplot(122)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [2]:
import numpy as np
import tensorflow as tf
from Modelling.Baselines.Data_Preprocessing import import_labeled_data,import_labeled_photos
print("1")

x,y=import_labeled_photos()
#X_train_=np.random.rand(31,31,16)
X_train_=x[0]
#y_train_=np.random.rand(31,1)
y_train_=y[0]
print("2")
batch=32

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(31*31, activation='relu'),
    tf.keras.layers.Dense(31*31, activation='relu'),
    tf.keras.layers.Dense(31*31, activation='relu'),
    tf.keras.layers.Dense(31*31, activation='relu'),
    tf.keras.layers.Dense(31*31, activation='relu'),
    tf.keras.layers.Dense(31*31, activation='sigmoid'),
    tf.keras.layers.Reshape((31,31))
])

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print("3")
num_epochs = 20
model.fit(X_train_, y_train_, epochs=num_epochs,batch_size=batch)

1
2
3
Epoch 1/20
1/1 [==============================] - 1s 828ms/step - loss: 6.1746 - accuracy: 0.3569
Epoch 2/20
1/1 [==============================] - 0s 62ms/step - loss: 5.5934 - accuracy: 0.3247
Epoch 3/20
1/1 [==============================] - 0s 62ms/step - loss: 5.3116 - accuracy: 0.3413
Epoch 4/20
1/1 [==============================] - 0s 47ms/step - loss: 6.0786 - accuracy: 0.3226
Epoch 5/20
1/1 [==============================] - 0s 47ms/step - loss: 5.1523 - accuracy: 0.3226
Epoch 6/20
1/1 [==============================] - 0s 62ms/step - loss: 4.1898 - accuracy: 0.3195
Epoch 7/20
1/1 [==============================] - 0s 47ms/step - loss: 6.4730 - accuracy: 0.3195
Epoch 8/20
1/1 [==============================] - 0s 47ms/step - loss: 4.8415 - accuracy: 0.3091
Epoch 9/20
1/1 [==============================] - 0s 47ms/step - loss: 5.9763 - accuracy: 0.2914
Epoch 10/20
1/1 [==============================] - 0s 62ms/step - loss: 5.6257 - accuracy: 0.2862
Epoch 11/20
1/1 [=====

In [8]:
import numpy as np
import tensorflow as tf
from Modelling.Baselines.Data_Preprocessing import import_labeled_data,import_labeled_photos
print("1")

x,y=import_labeled_photos()
#X_train_=np.random.rand(31,31,16)
#y_train_=np.random.rand(31,1)
x_train_data=x[0:2]
y_train_data=y[0:2]
x_test_data=x[3:5]
y_test_data=y[3:5]

X_train = tf.convert_to_tensor(x_train_data, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train_data, dtype=tf.float32)
X_test = tf.convert_to_tensor(x_test_data, dtype=tf.float32)
y_test = tf.convert_to_tensor(y_test_data, dtype=tf.float32)

print("2")
batch=32

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32,(3,3), activation='relu', input_shape=(200, 200, 108),padding='same'),
    #tf.keras.layers.Dense(31*31, activation='sigmoid'),
    tf.keras.layers.AveragePooling2D(pool_size=(3, 3),padding='same'),
    #tf.keras.layers.Conv2D(32*32, (2,2),  activation="relu",padding='same',),
    #tf.keras.layers.AveragePooling2D(pool_size=(2, 2),padding='same'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(200*200, activation='relu'),
    tf.keras.layers.Dense(7)
    #tf.keras.layers.Reshape(target_shape=(31, 31, 1))

])
model.summary()
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print("3")
num_epochs = 2
model.fit(X_train, y_train, epochs=num_epochs,batch_size=batch)

1
2


ResourceExhaustedError: OOM when allocating tensor with shape[143648,40000] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu [Op:Mul]

In [2]:
from segmentation_models.metrics import iou_score
from segmentation_models.losses import bce_jaccard_loss
import numpy as np
from segmentation_models import Unet
from keras.layers import Input, Conv2D
from keras.models import Model
import wandb
from wandb.keras import WandbCallback
import tensorflow as tf
from Modelling.Baselines.Data_Preprocessing import import_labeled_data,import_labeled_photos
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
# login
wandb.init(project="NN_for_pixels", entity="pds_project", name='segmentation_models_Unet')
print("1")

x,y=import_labeled_photos()
#X_train_=np.random.rand(31,31,16)
#y_train_=np.random.rand(31,1)
x_train_data=x[0:10]
y_train_data=y[0:10]
x_test_data=x[10:]
y_test_data=y[10:]

x_train = tf.convert_to_tensor(x_train_data, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train_data, dtype=tf.float32)
x_test = tf.convert_to_tensor(x_test_data, dtype=tf.float32)
y_test = tf.convert_to_tensor(y_test_data, dtype=tf.float32)

# define number of channels
N = x_train.shape[-1]

# define model
model = Unet(backbone_name='resnet34', encoder_weights=None, input_shape=(None, None, N))

model.summary()
model.compile('Adam', loss=tf.keras.losses.CategoricalCrossentropy(),  metrics=['accuracy'])

## fit model
#model.fit(
#    x=x_train,
#    batch_size=1,
#    epochs=100,
#    callbacks=[WandbCallback()],
#)
wandb.init(project="NN_for_pixels", entity="pds_project", name='segmentation_models_Unet_batch=5_loss=CategoricalCrossentropy')
model.fit(
    x=x_train,
    y=y_train,
    batch_size=5,
    epochs=1000,
    validation_data=(x_test, y_test),
    callbacks=[WandbCallback()],
)
wandb.finish()

1
Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 data (InputLayer)              [(None, None, None,  0           []                               
                                 108)]                                                            
                                                                                                  
 bn_data (BatchNormalization)   (None, None, None,   324         ['data[0][0]']                   
                                108)                                                              
                                                                                                  
 zero_padding2d_34 (ZeroPadding  (None, None, None,   0          ['bn_data[0][0]']                
 2D)                            108)                                                      

TypeError: Invalid keyword argument(s) in `compile()`: ({'learning_rate'},). Valid keyword arguments include "cloning", "experimental_run_tf_function", "distribute", "target_tensors", or "sample_weight_mode".

In [10]:
from datetime import datetime
datetime.now().strftime("%m/%d/%Y, %H:%M:%S")

'06/07/2022, 14:58:22'

In [ ]:
wandb.finish()

In [ ]:
from Modelling.Baselines.Data_Preprocessing import import_labeled_data,import_labeled_photos
from segmentation_models import Unet
x,y=import_labeled_photos()
#X_train_=np.random.rand(31,31,16)
#y_train_=np.random.rand(31,1)
x_train_data=x[0:11]
y_train_data=y[0:11]
x_test_data=x[11:]
y_test_data=y[11:]

In [3]:
from tensorflow import keras
import numpy as np

# data
(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()

train_images = np.expand_dims(train_images, axis=3) / 255.
test_images = np.expand_dims(test_images, axis=3) / 255.

print(train_images[0].shape)
print(train_labels[0])


# model
num_filters = 8
stride = 1
filter_size = 3
pool_size = 2

inputs = keras.Input((28, 28, 1,))
conv_layer = keras.layers.Conv2D(num_filters, filter_size, strides=stride)
tf.keras.layers.Conv3D(2, 3, activation='relu', input_shape=input_shape[1:])(x)

conv = conv_layer(inputs)
pool_layer = keras.layers.MaxPooling2D(pool_size=pool_size)
pool = pool_layer(conv)
flatten_layer = keras.layers.Flatten()
flat_features = flatten_layer(pool)
output = keras.layers.Dense(10, activation='softmax')(flat_features)

model = keras.Model(inputs, output)
model.compile(loss='categorical_crossentropy', metrics='accuracy')
model.summary()

# training
model.fit(
    train_images, keras.utils.to_categorical(train_labels),
    epochs=5,
    validation_data=(test_images, keras.utils.to_categorical(test_labels))
)

(28, 28, 1)
5
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 8)         80        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 8)        0         
 )                                                               
                                                                 
 flatten_1 (Flatten)         (None, 1352)              0         
                                                                 
 dense_6 (Dense)             (None, 10)                13530     
                                                                 
Total params: 13,610
Trainable params: 13,610
Non-trainable params: 0
___________________________________________